In [10]:
import torch
import torch.nn as nn

### <u>Compare functions one by one</u>

#### 1. MaxPool1d

In [47]:
#Create some random input
random_tensor = torch.rand(3, 4)
random_tensor

tensor([[0.5260, 0.2067, 0.6972, 0.6780],
        [0.7745, 0.1303, 0.6338, 0.8962],
        [0.0936, 0.5606, 0.8696, 0.9967]])

In [48]:
#define the pytorch equivalent
pytorch_max_pool_fn = nn.MaxPool1d(kernel_size=2, stride=1, padding=0,dilation=1, return_indices=False, ceil_mode=False)
output_from_pytorch = pytorch_max_pool_fn(random_tensor)
output_from_pytorch

tensor([[0.5260, 0.6972, 0.6972],
        [0.7745, 0.6338, 0.8962],
        [0.5606, 0.8696, 0.9967]])

In [49]:
#define my custom implementation of maxPool1d ( assumes that x is 2D (C,W) shape)
def my_MaxPool1d(x, kernel_size=2, stride=1, padding=0, dilation=1):
    result = []
    for row in x:
        maxPoolForRow = []
        #k represents start point of the kernel
        for k in range(0, len(row) - kernel_size + 1, stride):
            maxPoolForRow.append(max(row[k:k + kernel_size]))
        result.append(maxPoolForRow)
    result = torch.tensor(result)
    return result


output_from_my_implementation = my_MaxPool1d(random_tensor)
output_from_my_implementation

tensor([[0.5260, 0.6972, 0.6972],
        [0.7745, 0.6338, 0.8962],
        [0.5606, 0.8696, 0.9967]])

In [50]:
#Assert that both are the same
torch.equal(output_from_pytorch, output_from_my_implementation)

True

#### 2. AvgPool1d

In [51]:
#Create some random input
random_tensor = torch.rand(3, 4)
random_tensor

tensor([[0.3352, 0.0846, 0.9417, 0.8539],
        [0.4941, 0.7751, 0.7935, 0.9459],
        [0.8634, 0.1323, 0.0889, 0.7605]])

In [52]:
#define the pytorch equivalent
pytorch_avg_pool_fn = nn.AvgPool1d(kernel_size=2, stride=1, padding=0,ceil_mode=False, count_include_pad=True)
output_from_pytorch = pytorch_avg_pool_fn(random_tensor)
output_from_pytorch


tensor([[0.2099, 0.5131, 0.8978],
        [0.6346, 0.7843, 0.8697],
        [0.4978, 0.1106, 0.4247]])

In [53]:
#define my custom implementation of avgPool1d ( assumes that x is 2D (C,W) shape)
def my_AvgPool1d(x, kernel_size=2, stride=1, padding=0, dilation=1):
    result = []
    for row in x:
        maxPoolForRow = []
        #k represents start point of the kernel
        for k in range(0, len(row) - kernel_size + 1, stride):
            maxPoolForRow.append(sum(row[k:k + kernel_size])/kernel_size)
        result.append(maxPoolForRow)
    result = torch.tensor(result)
    return result


output_from_my_implementation = my_AvgPool1d(random_tensor)
output_from_my_implementation

tensor([[0.2099, 0.5131, 0.8978],
        [0.6346, 0.7843, 0.8697],
        [0.4978, 0.1106, 0.4247]])

In [54]:
#Assert that both are the same
torch.equal(output_from_pytorch, output_from_my_implementation)

True

#### 3. Conv1d

In [65]:
#Loading the given filter and input tensors
pixel_input = torch.load("./pixel_input.pt") #Shape (1,1,32)
filter = torch.load("./filter.pt") #(Shape 3,1,1)

In [66]:
#define pytorch equivalent
pytorch_conv1d_output = nn.functional.conv1d(pixel_input, filter, bias = None,stride = 1, padding = 0, dilation = 1, groups = 1)

#Shape is (1,3,32)

In [67]:
#define custom implementation
def my_Conv1d(inp, filter, stride=1, padding=0, dilation=1, groups = 1):
    result = []

    for f in filter:
        #as f in this case is 1x1 ( but it could in theory be something like 1 x k)
        f = f[0]
        batch_row = []
        for batch in inp: #(1,32)
            conv_row = []
            for channel in batch: #(32,)
                #convolve f over this channel array using this filter by using dot product to convolve!
                for f_ind in range(0, len(channel) - len(f) + 1, stride):
                    conv_row.append(torch.dot(f, channel[f_ind: f_ind + len(f)]))
            batch_row.append(conv_row)
        result.append(batch_row)

    result = torch.tensor(result)

    #permute it so that the shapes are correct.
    result = result.permute(1,0,2)
    return result

my_conv1d_output = my_Conv1d(pixel_input, filter)

In [68]:
#Assert that both are the same
torch.equal(pytorch_conv1d_output, my_conv1d_output)

True

#### 4. Sigmoid



In [69]:
#Create some random input
random_tensor = torch.rand(2, 2)
random_tensor

tensor([[0.9921, 0.1134],
        [0.2238, 0.0080]])

In [70]:
#define the pytorch equivalent
pytorch_sigmoid_fn = nn.Sigmoid()
output_from_pytorch = pytorch_sigmoid_fn(random_tensor)
output_from_pytorch


tensor([[0.7295, 0.5283],
        [0.5557, 0.5020]])

In [71]:
#define my custom implementation of maxPool1d ( assumes that x is 2D (C,W) shape)
def my_Sigmoid(x):
    return 1/(1+torch.exp(-x))

output_from_my_implementation = my_Sigmoid(random_tensor)
output_from_my_implementation

tensor([[0.7295, 0.5283],
        [0.5557, 0.5020]])

In [72]:
#Assert that both are the same
torch.equal(pytorch_conv1d_output, my_conv1d_output)

True

#### 5. BatchNorm1d